In [81]:
import matplotlib.pyplot as plt
from msmfcode.core.config import *
from msmfcode.evaluation.plot import plot_error
from msmfcode.evaluation.data import load_experimental_data, load_experimental_config
from msmfcode.models.cann import MSMFSingleCAN, MSMFMultiCAN, MSMFGridCAN, SSSFCAN

plt.rcParams.update({'font.size': 21})
plt.rcParams['text.usetex'] = True

ERROR_LABELS = {Metric.POS_ERROR_MEAN: 'Median positional decoding error $E_{pos}^{\\tilde{\mu}}$ (m)',
                Metric.POS_ERROR_STD: 'Std. deviation of positional error (m)',
                Metric.POS_ERROR: 'Positional error (m)',
                Metric.POS_ERROR_NUM_CAT: 'Number of catastrophic errors',
                Metric.MEAN_FIELD_ACTIVITY: 'Mean field activity',
                Metric.PERC_CORRECT_FIELDS: 'Proportion of correct fields',
                Metric.AVG_NUM_FIELDS_PER_NEURON: 'Avg. number of fields (per neuron)',
                Metric.PERC_UNIQUE_FIELD_COMBINATIONS: 'Proportion of unique field combinations',
                Metric.ACTIVITY_FALSE_POSITIVES_NUM: 'Number of bins with false positive field',
                Metric.ACTIVITY_FALSE_NEGATIVES_NUM: 'Number of bins with false negative field',
                }


In [82]:
import re
def replace_params(word, data):
    regex = r'@([a-z_A-Z0-9]+)@'
    new_word = word
    for match in re.finditer(regex, word):
        param_name = match.groups()[0]
        new_word = new_word.replace(f'@{param_name}@', str(data[param_name]))
    return new_word


# Load experimental data
def plot_multi(plot_type_id, new_figure=False):
    data = []
    ax = None
    for i_plot, plot_type in enumerate(plot_types):
        for i_exp, experiment_num in enumerate(experiment_range):
            curr_data = load_experimental_data(ExperimentType.EVALUATION, network_type, experiment_num,
                                               experiment_id=experiment_id)
            data.append(curr_data)
            curr_config = load_experimental_config(ExperimentType.EVALUATION, network_type, experiment_num,
                                                   experiment_id=experiment_id)

            x_data = curr_data[curr_config['evaluated_parameter']]
            y_data = curr_data[plot_type]
            # y_label = ERROR_LABELS[plot_type if i_plot == 0 else Metric.POS_ERROR]
            if plot_type[::-1].split('_', 1)[1][::-1] not in ERROR_LABELS.keys():
                y_label = 'Placeholder'
            else:
                y_label = ERROR_LABELS[plot_type[::-1].split('_', 1)[1][::-1]]
            last_exp = i_exp == num_exp-1 and plot_type == plot_types[-1]
            i_total = i_plot*len(experiment_range)+i_exp

            labels[i_total] = replace_params(labels[i_total], curr_config)

            # labels[i_total] = labels[i_total].replace('$num_neurons$', curr_config[NUM_NEURONS])
            # labels[i_total] = labels[i_total].replace('$env_length$', curr_config[ENV_LENGTH])

            line_style_id = 0 if not vary_line_style else i_total
            color_id = plot_type_id if vary_line_style else i_total+plot_num_offset

            plot_error(network_type, x_data, y_data, experiment_id, experiment_num, x_label=x_label, y_label=y_label,
                       img_name='_'.join(plot_types), plot_type=plot_type_name, label=labels[i_total], color_id=color_id, line_style_id=line_style_id,
                       plot_fitted_poly=fit_polys, max_poly=max_polys[i_total], plot_min_y=plot_min_y[i_total],
                       save_plot=(last_exp and save_plot), use_all_plot_names=True, second_plot=last_exp and second_axis,
                       show_legend=show_legend, show_plot=False, clear_plot=False, plot_size=plot_size, new_figure=(i_plot==i_exp==0) and new_figure,legend_loc=legend_loc)

### General Config

In [83]:
# plt.rcParams['figure.dpi'] = 100

PLOT_FILE_TYPE = 'pdf'
# experiment_id = '3-2-a'
# experiment_id = '2-1-d'
experiment_id = '2-4-a'
legend_loc = 'left'
# plot_type_name = 'circles'
plot_type_name = 'line-circles'
# img_name = 'field-ratio-threshold'
# img_name = 'Patt-median-pos-error'
# img_name = 'Patt-median-cat-error'
# img_name = 'Patt-unique-comb'
# img_name = 'Patt-false-pos'
# img_name = 'max-field-size'
img_name = 'lesion-study_all'
plot_size = (24, 14)
plot_num_offset = 0
experiment_range = []
second_axis = False
save_plot = False
show_legend = True
vary_line_style = True
fit_polys = False
plot_min_y_global = False
plot_types = [
    Metric.POS_ERROR_MEAN+'_median',
    # Metric.POS_ERROR_NUM_CAT+'_median',
    # Metric.AVG_NUM_FIELDS_PER_NEURON+'_mean',
    # Metric.ACTIVITY_FALSE_NEGATIVES_NUM+'_median',
    # Metric.ACTIVITY_FALSE_POSITIVES_NUM+'_mean',
    # Metric.PERC_UNIQUE_FIELD_COMBINATIONS+'_median',
    # Metric.PERC_CORRECT_FIELDS+'_mean'
              ]
# x_label = 'Probability of neuron attractor participation - $P_{att}$'
# x_label = 'Field ratio threshold - $\\tau_{fs}$'
x_label = 'Proportion of drop-out neurons - $P_{dro}$'
# x_label = 'Accumulated field size per neuron - $\\bar{\Sigma}_{fs}$ (m)'

### 1. SSSF

In [84]:
##  General Config
# PLOT_FILE_TYPE = 'pdf'
# experiment_id = '2-4-a'
# second_axis = False
# save_plot = True
# show_legend = True
# fit_polys = False
# plot_size = (12, 9)

## Individual Config
network_type = SSSFCAN
experiment_range = [1]
num_exp = len(experiment_range)
plot_num_offset = 0

max_polys = [3, 3]
plot_min_y = [False, False]

labels = ['SSSF [$N_{neu}=@num_neurons@$]']
# labels = ['V-MSMF [N=$num_neurons$, L=$env_length$]']
# labels = ['V-MSMF (mean error) [N=$num_neurons$, L=$env_length$]',
#           'V-MSMF (correct fields) [N=$num_neurons$, L=$env_length$]']

# labels = ['500 total neurons', '1000 total neurons']
# labels = ['1D Variable-MSMF (200 m)', '1D Variable-MSMF (400 m)']
# labels = ['1D Fixed-MSMF (mean)', '1D Fixed-MSMF (std. dev.)']
# plot_types = [Metric.POS_ERROR_MEAN+'_median', Metric.PERC_CORRECT_FIELDS+'_mean']
# plot_types = [Metric.POS_ERROR_MEAN+'_mean', Metric.AVG_NUM_FIELDS_PER_NEURON+'_mean']
# plot_types = [Metric.POS_ERROR_MEAN+'_mean']
# x_label = 'Max. total coverage of all fields per neuron (m)'
# x_label = 'Percentage of neurons per attractor - f'
# x_label = 'Total number of neurons - N'
# x_label = 'Number of dead neurons - $N_{d}$'

plot_multi(0, new_figure=True)

## 2. Grid-CAN

In [85]:
## General Config
# PLOT_FILE_TYPE = 'pdf'
# experiment_id = '2-4-a'
# second_axis = False
# save_plot = True
# show_legend = True
# fit_polys = False
# plot_size = (12, 9)

## Individual Config
network_type = MSMFGridCAN
# plot_num_offset = 0
plot_num_offset += len(experiment_range)
experiment_range = [1, 2, 3, 4, 5]
num_exp = len(experiment_range)

max_polys = [6, 3, 3, 3, 3]
plot_min_y = [False for i in experiment_range]

models = ['G-Opt-1', 'G-Opt-2', 'G-Opt-3', 'G-Opt-4', 'G-Opt-5']

labels = [m + ' [$N_{neu}=@num_neurons@$, $N_{mod}=@num_modules@$, $N_{nmod}=@num_neurons_per_module@$]' for m in models]

# labels = ['V-MSMF [N=$num_neurons$, L=$env_length$]']
# labels = ['500 total neurons', '1000 total neurons']
# labels = ['1D Variable-MSMF (200 m)', '1D Variable-MSMF (400 m)']
# labels = ['1D Fixed-MSMF (mean)', '1D Fixed-MSMF (std. dev.)']

# plot_types = [Metric.POS_ERROR_MEAN+'_median', Metric.PERC_CORRECT_FIELDS+'_mean']
# plot_types = [Metric.POS_ERROR_MEAN+'_mean']

# x_label = 'Percentage of neurons per attractor - f'
# x_label = 'Total number of neurons - N'
# x_label = 'Number of dead neurons - $N_{d}$'

plot_multi(1)


## 3. F-MSMF

In [86]:
## General Config
# PLOT_FILE_TYPE = 'pdf'
# experiment_id = '2-4-a'
# second_axis = False
# save_plot = True
# show_legend = True
# fit_polys = False
# plot_size = (12, 9)

# Individual Config
network_type = MSMFMultiCAN
# plot_num_offset = 0
plot_num_offset += len(experiment_range)
# start_id = 16
# experiment_range = list(range(start_id, start_id+5))
experiment_range = [1, 2, 6, 7, 3]
num_exp = len(experiment_range)

max_polys = [3 for i in experiment_range]
plot_min_y = [False for i in experiment_range * (second_axis + 1)]
# labels = ['F-MSMF [$N_{neu}=@num_neurons@$, $N_{AL}=[@nmin@, @nmed@, @nmax@]$, $W_{inh}=@J0@, W_{exc}=@J1@, I_{bck}=@I@$]' for i in experiment_range]

models = ['F-Org-1', 'F-Org-2 ', 'F-Opt-1', 'F-Opt-2', 'F-Opt-3']

labels = [m + ' [$N_{neu}=@num_neurons@$, $N_{AL}=[@nmin@, @nmed@, @nmax@]$]' for m in models]

# labels = ['V-MSMF [N=$num_neurons$, L=$env_length$]']
    # labels = ['F-MSMF (mean error) [N=$num_neurons$, L=$env_length$]',
    #           'F-MSMF (correct fields) [N=$num_neurons$, L=$env_length$]']
# labels = ['500 total neurons', '1000 total neurons']
# labels = ['1D Variable-MSMF (200 m)', '1D Variable-MSMF (400 m)']
# labels = ['1D Fixed-MSMF (mean)', '1D Fixed-MSMF (std. dev.)']

# plot_types = [Metric.POS_ERROR_MEAN+'_median', Metric.PERC_CORRECT_FIELDS+'_mean']
# plot_types = [Metric.POS_ERROR_MEAN+'_mean', Metric.AVG_NUM_FIELDS_PER_NEURON+'_mean']
# plot_types = [Metric.POS_ERROR_MEAN+'_mean']

# x_label = 'Max. total coverage of all fields per neuron (m)'
# x_label = 'Percentage of neurons per attractor - f'
# x_label = 'Total number of neurons - N'
# x_label = 'Number of dead neurons - $N_{d}$'

print(plot_types)

plot_multi(2)

# # 2nd network
# labels = ['F-MSMF [$N_{neu}=@num_neurons@$, $N_{AL}=[@nmin@, @nmed@, @nmax@]$, $W_{inh}=@J0@, W_{exc}=@J1@, I_{bck}=@I@$]' for i in experiment_range]
# experiment_range = list(range(start_id+5, start_id+10))
# plot_multi(1, new_figure=False)
#
# # # 3rd network
# experiment_range = list(range(start_id+10, start_id+15))
# labels = ['F-MSMF [$N_{neu}=@num_neurons@$, $N_{AL}=[@nmin@, @nmed@, @nmax@]$, $W_{inh}=@J0@, W_{exc}=@J1@, I_{bck}=@I@$]' for i in experiment_range]
# plot_multi(2, new_figure=False)

# plt.show()

['pos_error_mean_median']


## 4. D-MSMF

In [87]:
## General Config
# PLOT_FILE_TYPE = 'pdf'
# experiment_id = '2-4-a'
# second_axis = False
# save_plot = True
# show_legend = True
# fit_polys = False
# plot_size = (12, 9)

## Individual Config
network_type = MSMFSingleCAN
# plot_num_offset = 0
plot_num_offset += len(experiment_range)
experiment_range = [1, 2]
num_exp = len(experiment_range)
max_polys = [3, 3]
plot_min_y = [False for i in experiment_range]
# labels = ['V-MSMF [$N=@num_neurons@$]']

models = ['D-Org-1', 'D-Opt-1']

# labels = ['V-MSMF [N=@num_neurons$, L=$env_length$]']
labels = [m + r' [$N_{neu}=@num_neurons@$, $\alpha=@alpha@$, $\theta=@theta@$, $\bar{\Sigma}_{fs}=@max_field_sizes@$]' for m in models]
# labels = ['V-MSMF (mean error) [N=$num_neurons$, L=$env_length$]',
#           'V-MSMF (correct fields) [N=$num_neurons$, L=$env_length$]']
# labels = ['500 total neurons', '1000 total neurons']
# labels = ['1D Variable-MSMF (200 m)', '1D Variable-MSMF (400 m)']
# labels = ['1D Fixed-MSMF (mean)', '1D Fixed-MSMF (std. dev.)']

# plot_types = [Metric.POS_ERROR_MEAN+'_median', Metric.PERC_CORRECT_FIELDS+'_mean']
# plot_types = [Metric.POS_ERROR_MEAN+'_mean', Metric.AVG_NUM_FIELDS_PER_NEURON+'_mean']
# plot_types = [Metric.POS_ERROR_MEAN+'_mean']

# x_label = 'Max. total coverage of all fields per neuron (m)'
# x_label = 'Percentage of neurons per attractor - f'
# x_label = 'Total number of neurons - N'
# x_label = 'Number of dead neurons - $N_{d}$'

# plot_multi(0, new_figure=True)
plot_multi(3)

# plt.show()

## Run Plotting

In [88]:
# import re
# def replace_params(word, data):
#     regex = r'@([a-z_A-Z0-9]+)@'
#     new_word = word
#     print(word)
#     for match in re.finditer(regex, word):
#         param_name = match.groups()[0]
#         print(param_name)
#         new_word = new_word.replace(f'@{param_name}@', str(data[param_name]))
#         print(new_word)
#         print()
#     return new_word
#
#
# # Load experimental data
# data = []
# ax = None
# for i_plot, plot_type in enumerate(plot_types):
#     for i_exp, experiment_num in enumerate(experiment_range):
#         curr_data = load_experimental_data(ExperimentType.EVALUATION, network_type, experiment_num,
#                                            experiment_id=experiment_id)
#         data.append(curr_data)
#         curr_config = load_experimental_config(ExperimentType.EVALUATION, network_type, experiment_num,
#                                                experiment_id=experiment_id)
#
#         x_data = curr_data[curr_config['evaluated_parameter']]
#         y_data = curr_data[plot_type]
#         print(y_data)
#         # y_label = ERROR_LABELS[plot_type if i_plot == 0 else Metric.POS_ERROR]
#         y_label = ERROR_LABELS[plot_type[::-1].split('_', 1)[1][::-1]]
#         last_exp = i_exp == num_exp-1 and plot_type == plot_types[-1]
#         i_total = i_plot*len(experiment_range)+i_exp
#
#         labels[i_total] = replace_params(labels[i_total], curr_config)
#
#         # labels[i_total] = labels[i_total].replace('$num_neurons$', curr_config[NUM_NEURONS])
#         # labels[i_total] = labels[i_total].replace('$env_length$', curr_config[ENV_LENGTH])
#
#         plot_error(network_type, x_data, y_data, experiment_id, experiment_num, x_label=x_label, y_label=y_label,
#                    img_name='_'.join(plot_types), plot_type='circles', label=labels[i_total], color_id=i_total+plot_num_offset,
#                    plot_fitted_poly=fit_polys, max_poly=max_polys[i_total], plot_min_y=plot_min_y[i_total],
#                    save_plot=(last_exp and save_plot), use_all_plot_names=True, second_plot=last_exp and second_axis,
#                    show_legend=show_legend, show_plot=False, clear_plot=False, plot_size=plot_size)

In [89]:
# Save Figure
# img_file_path = join(EXPERIMENT_FOLDERS[NetworkType.GENERAL], EXPERIMENT_SUBFOLDERS[FileType.FIGURE],
#                          f'{NetworkType.GENERAL}_{experiment_id}_{img_name}.{PLOT_FILE_TYPE}')
# plt.savefig(img_file_path)

In [90]:
plt.show()